Welcome to lesson 9 of the Noisebridge Python class! ([Noisebridge Wiki](https://www.noisebridge.net/wiki/PyClass) | [Github](https://github.com/audiodude/PythonClass))

In this lesson, we will learn about using APIs in Python, especially those for interacting with Large Language Models (LLMs, also known as "AI").

You will learn:

- What an API is
- How to get and store an API "key"
- The basics of _RESTful_ APIs
- How to use a library or SDK to help with API tasks
- How to use multiple APIs in a single project

Let's get started!


## APIs

_If you remember back all the way to lesson 1, the very first thing we did in this class was post a status to the social media site Mastodon using its API._

API stands for "Application Programming Interface". It is generally any mechanism by which your program can interact with another program, application, system or server. APIs can be **local** or **remote**. A local API, for example, is the Python Database API. It is a standard way of interacting with a database, but the code runs on the computer where you are running your Python code. On the other hand, a remote API lets you interact with a program or server that is running on a different computer than yours. For example, you could use the [Google Maps API](https://developers.google.com/maps) to query restaurants in the vincinity of a coordinate that your program generates.

In this lesson, we will be discussing remote APIs, as a way of interacting with a remote service or application.

An API defines the way your program interacts with the program providing the API. This could include mechanisms for authenticating yourself, what kind of network calls you should make to invoke a remote function, the format of data that will be returned in a successful response, what to expect if there is an error, and more.

There are lots of services on the web that you might wish to interact with in your programs, such as:

1. Google Maps
1. Google Drive
1. Dropbox
1. YouTube
1. Spotify
1. Amazon Web Services
1. Twitter (not anymore)
1. Reddit (not anymore)


## Project: New Video Ideas

Let's create a project that works like this: we give it a YouTube channel (like [Mark Rober](https://www.youtube.com/MarkRober) for example) and it uses AI to suggest some new video ideas/titles based on the videos already uploaded to the channel.

Let's think about the high-level "algorithm" that we would use to implement this:

1. Receive a YouTube channel id as input.
1. Use the YouTube API to get a list of videos that were uploaded to that channel.
1. Extract the video title for each video in the list.
1. Format an AI prompt using the titles from the previous step ("Dear AI, please look at these titles and suggest new ones")
1. Contact the LLM via its API to provide the prompt
1. Read back the response from the LLM API and display the results to the user


### API keys

Almost every intersting API operates using **API keys**. An API key is a token that you present to the API when making a request, so that the program on the other end can identify you (authentication) and decide which resources you're allowed to access (authorization). APIs commonly have **rate limits** which restrict how many requests you can make, like "100 requests per minute" or "1000 requests per day".

We will need an API key for YouTube as well as Google Gemini (the LLM we will be using for this project).

- Here's how to get a key for [Google Gemini](https://ai.google.dev/gemini-api/docs/api-key).
- Here's a significantly more complicated description of how to get [a key for YouTube](https://developers.google.com/youtube/v3/getting-started).


### Keeping API keys safe

An API key is a credential, like a password, and should be kept just as safe. Unlike passwords, we usually need to provide the key to our programs in order to make the API calls we need. You should never store an API key in a version control system like git, but it's usually okay to have them sitting around on your computer.

In this directory, we have a file called `keys.py.example`. It contains a few variables that are set to empty strings. In the case that you want to share your program, the person you share it with will need their own API keys. This file is a standard-ish way of indicating that the person should move the file `keys.py.example` to `keys.py` and properly populate it.

We will have our own copy of `keys.py` that we will NOT check into git. Here we will store our keys and import them with code like:


In [ ]:
from keys import YOUTUBE_API_KEY

## Getting user input

We can get input from the user using the built-in `input()` method. This will cause the program to pause and wait for the user to type some characters and hit `ENTER`. It even works in a Jupyter notebook.


In [ ]:
print('Enter the YouTube URL of the channel: ')
url = input()
handle = url.split('/')[-1]  # Take everything after the last /
print(handle)

## Calling APIs

Most APIs nowadays are what we call [RESTful APIs](https://blog.postman.com/rest-api-examples/). There are a lot of minutae around what does and does not qualify as RESTful, but it's not terribly important. The key feature of a RESTful API is that it is accessed over HTTP (the same protocol that web browsers use). This is convenient because HTTP requests are easy to make from Python, and has the added benefit that in many cases you can browse or debug API responses by simply using your web browser.

We can call a RESTful API simply by making HTTP requests and parsing the returned data (which is usually [JSON](https://www.json.org/json-en.html) but sometimes [XML](https://developer.mozilla.org/en-US/docs/Web/XML/XML_introduction)).

However, in the case of larger APIs provided by a big company or service, there is likely a library we can download/install that will make things simpler by abstracting the HTTP requests and providing objects that represent the most interesting data structures. It's usually easier to use a library (sometimes called an SDK) provided by the service.

In this case we install the `google-api-python-client` library (which I've installed in sfpythonlab.com) and use that. This code comes directly from the Google/YouTube docs on how to call this precise method.


In [ ]:
from pprint import pprint

import googleapiclient.discovery

from keys import YOUTUBE_API_KEY

youtube = googleapiclient.discovery.build("youtube",
                                          "v3",
                                          developerKey=YOUTUBE_API_KEY)

request = youtube.channels().list(part="contentDetails", forHandle=handle)
channel_response = request.execute()

pprint(channel_response)

This same response is available in a web browser, provided I give my API key: https://www.googleapis.com/youtube/v3/channels?forHandle=numberphile&part=contentDetails&key=


_Now that we have a channel response from the YouTube API, can you identify how to access the id of the "uploads" playlist that contains all of the channel's uploads?_


Once we have the playlist id of the uploads of our channel, we can use the `PlaylistItems` endpoint to list the videos in that playlist. How do I know this? I dug around in the [API documentation](https://developers.google.com/youtube/v3/docs), clicking on various headers and reading the technical descriptions until I found something that looked like what I wanted.

We will need to use **pagination** to get the videos for the channel, since the YouTube API (sensibly) won't return them all at once. Basically, if we make a call to `PlaylistItems`, and we get back a `nextPageToken`, we can make a subsequent call using that token to get the next page of results. Keep doing that until there is no `nextPageToken`.


In [ ]:
all_titles = []

uploads_id = channel_response['items'][0]['contentDetails']['relatedPlaylists'][
    'uploads']

# The API will ignore a pageToken of None, so we can use it to start the pagination
page_token = None

repeats = 2  # Get up to 100 titles
while repeats > 0:
    playlist_items = youtube.playlistItems().list(
        part='snippet',
        playlistId=uploads_id,
        maxResults=50,
        pageToken=page_token).execute()

    for item in playlist_items['items']:
        video_title = item['snippet']['title']
        all_titles.append(video_title)

    # Check if there are more pages
    if 'nextPageToken' in playlist_items:
        page_token = playlist_items['nextPageToken']
        repeats -= 1
    else:
        break

## Calling our LLM

LLMs, as implemented in most major services, operate on a series of prompts and responses. You are probably familiar with this in the case of ChatGPT. In order to get a good response from our LLM (Google Gemini), we must be sure we are offering a good prompt. Here's the prompt I would suggest:

```
The following is a list of titles of videos in my YouTube channel. Based on this list, please suggest some titles for new videos:

<LIST>
```


In [ ]:
import google.generativeai as genai

from keys import GEMINI_API_KEY

base_prompt = (
    'The following is a list of titles of videos in my YouTube channel. '
    'Based on this list, please suggest some titles for new videos:\n\n{list}')

prompt = base_prompt.format(list='\n'.join(all_titles))

genai.configure(api_key=GEMINI_API_KEY)
model = genai.GenerativeModel('gemini-2.0-flash')

llm_response = model.generate_content(prompt)

from IPython.display import Markdown

display(Markdown(llm_response.text))

## Closing remarks

This is a cool example where in about 80 lines of code we've grabbed data from YouTube and fed it into AI for interesting results. Really, the majority of the work here isn't writing the Python code. It's reading the API docs to figure out which endpoints correspond to the data we want, and how to call those endpoints. You'll get better and better at that the more you work with APIs, because there are some common patterns that they follow nowadays.
